In [1]:
# mike babb
# 2024 06 28
# what streets start and stop?
# step 02: export street nodes

In [2]:
# standard
import os

In [3]:
# external
from itertools import combinations, product
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
import shapely
from shapely.geometry import LineString, Point
from shapely import line_merge


In [4]:
# custom
from geodataio.geo_operations import points2distance, calculate_initial_compass_bearing

# load the streetnetwork

In [5]:
# file path
file_path = 'H:/project/seattle_streets/data/' 

In [6]:
file_name = 'Street_Network_Database_Complete.gpkg'

In [7]:
fpn = os.path.join(file_path, file_name)

In [8]:
gdf = gpd.read_file(filename = fpn)

# GENERATE STREET END VERTICES - THESE WILL BE HELPFUL WITH IDENTIFICATION IN SUBSEQUENT STEPS

In [9]:
def generate_street_end_vertices(gdf:gpd.GeoDataFrame):    
    
    
    # export the vertex of each street end. 
    gdf['start_coord'] = gdf['geometry'].map(lambda x: x.coords[0])
    
    gdf['end_coord'] = gdf['geometry'].map(lambda x: x.coords[-1])
    
    f_node_gdf = gdf[['f_intr_id', 'start_coord']].copy().rename(columns = {'f_intr_id':'node_id',
                                                                           'start_coord':'coord'})
    t_node_gdf = gdf[['t_intr_id', 'end_coord']].copy().rename(columns = {'t_intr_id':'node_id',
                                                                         'end_coord':'coord'})
    
    node_df = pd.concat(objs = [f_node_gdf, t_node_gdf]).drop_duplicates()
    
    # make a gdf
    node_gdf = gpd.GeoDataFrame(data = node_df, geometry =
                                node_df['coord'].map(lambda x: Point(x)), crs = 4326)
    
    node_gdf = node_gdf.drop(labels = ['coord'], axis = 1)

    # remove the start and end coordinates
    gdf = gdf.drop(labels = ['start_coord', 'end_coord'], axis = 1)
    
    return gdf, node_gdf

In [10]:
# can we generate the vertices? yes... but let's check what type of geometry we are working with
gdf['geometry'].map(lambda x: x.geom_type).value_counts()

geometry
MultiLineString    34378
Name: count, dtype: int64

In [11]:
# everything is a multiline string??? is that necessary?

In [12]:
test_gdf = gdf.explode()

In [13]:
test_gdf.shape

(34378, 38)

In [14]:
# nope! if MultiLineStrings were necessary, the number of records would increase. Let's use the .explode() function to convert things to a LineString

In [15]:
gdf = test_gdf.copy()

In [16]:
gdf['geometry'].geom_type.value_counts()

LineString    34378
Name: count, dtype: int64

In [17]:
gdf, node_gdf = generate_street_end_vertices(gdf = gdf)

In [18]:
output_file_name = 'Street_Network_Nodes.gpkg'

In [19]:
ofpn = os.path.join(file_path, output_file_name)

In [20]:
node_gdf.to_file(filename = ofpn)

# import the classified streets - this classifies streets without a classification. 

In [21]:
file_name = 'blank_street_type_modified.xlsx'

In [22]:
fpn = os.path.join(file_path, file_name)

In [23]:
blank_street_type_df = pd.read_excel(io = fpn)

In [24]:
blank_street_type_df.head()

,ord_stname_concat,ord_street_type_fix
0,NE SUNRISE VISTA,ST
1,NW ESPLANADE,ST
2,BROADWAY E,ST
3,FAUNTLEE CREST SW,ST
4,NE FOREST VISTA,ST


In [25]:
gdf = pd.merge(left = gdf, right = blank_street_type_df, how = 'left')

In [26]:
gdf.loc[gdf['ord_street_type'].isna(), 'ord_street_type'] = gdf.loc[gdf['ord_street_type'].isna(), 'ord_street_type_fix']

In [27]:
gdf['ord_street_type'].unique()

array(['AVE', 'CT', 'ST', 'PL', 'LN', 'RD', 'DR', 'WAY', 'PKWY', 'BLVD',
       'CIR', 'TER', 'AL', 'RP', 'LOOP', 'SR', 'IS', 'OP', 'TRL', 'RR',
       'BR', 'WKWY', 'ET', 'STCR', 'VIEW', 'FLYOVER', 'HWY', 'RN', 'PZ',
       'MALL'], dtype=object)

In [28]:
gdf = gdf.drop(labels = ['ord_street_type_fix'], axis = 1)

# keep only streets in Seattle

In [29]:
gdf = gdf.loc[(gdf['l_city'] == 'SEATTLE') |
(gdf['r_city'] == 'SEATTLE'), :].copy()

# WRITE THE FULL SEATTLE STREETS TO DISK

In [30]:
output_file_name = 'Street_Network_Database_Seattle_Full.gpkg'

In [31]:
ofpn = os.path.join(file_path, output_file_name)

In [32]:
gdf.to_file(filename = ofpn, driver = 'GPKG', index = False)

# EXPORT THE INDIVIDUAL STREET PORTIONS TO MAKE THE DIFFERENT SECTIONS

In [33]:
# after performing some analysis in GIS, we can quickly identify the central streets

In [34]:
file_name = 'Street_Network_Database_Seattle_Central_Streets.csv'

In [35]:
fpn = os.path.join(file_path, file_name)

In [36]:
cs_df = pd.read_csv(filepath_or_buffer=fpn)

In [37]:
cs_df = cs_df.drop(labels = ['fid'], axis =  1)

In [38]:
# working_gdf
# winnow down the list of streets to get what we're looking for 

q_str = 'snd_feacode in (1, 5) and st_code in (0, ) and city_portion != \'WB\''
wgdf = gdf.query(expr = q_str)

In [39]:
output_file_path = 'H:/project/seattle_streets/data/city_section'

if not os.path.exists(output_file_path):
    os.makedirs(output_file_path)

data_list = []
geom_list = []
for cp in wgdf['city_portion'].unique().tolist():
    print(cp)
    if cp == ' ':
        temp_geoms = wgdf.loc[(wgdf['snd_id'].isin(cs_df['snd_id'])) &
        (wgdf['city_portion'] == cp), 'geometry']
    else:
        temp_geoms = wgdf.loc[wgdf['city_portion'] == cp, 'geometry']
    
    
    # # create the concave hull
    # point_cloud = []
    # for tg in temp_geoms:
    #     for coords in tg.coords:
    #         point_cloud.append(Point(coords))
    # point_cloud = shapely.unary_union(geometries = point_cloud)
    # ccv_hull = shapely.concave_hull(geometry = point_cloud)    
    # temp_list = [cp, 'concave']
    # data_list.append(temp_list)
    # geom_list.append(ccv_hull)

    # create the convex hull
    geoms = shapely.unary_union(geometries = temp_geoms)
    cvx_hull = shapely.convex_hull(geometry = geoms)
    temp_list = [cp, 'convex']
    data_list.append(temp_list)
    geom_list.append(cvx_hull)   
    
    
    
    

S
E
NW
N
 
NE
SW
W


In [40]:
output_gdf = gpd.GeoDataFrame(data = data_list, columns = ['city_portion', 'hull_type'], geometry = geom_list, crs = 'epsg:4326')

In [41]:
output_file_path = 'H:/project/seattle_streets/data/'

In [42]:
file_name = 'city_sections.gpkg'

In [43]:
ofpn = os.path.join(output_file_path, file_name)

In [44]:
output_gdf.to_file(filename = ofpn, driver = 'GPKG', index = False)

# REMOVE "STREETS" THAT ARE REALLY SHORT TURNOUTS AND WALKING PATHS

AL: Alley
TRL: Trail
CIR: Circle
TER: Terrace
OP: Overpass
RP: Ramp
ET: Extension
RN: TURN




In [45]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_portion', 'geometry'],
      dtype='object')

In [46]:
# remove streets of the following type
street_type_to_remove = [ 'AL', 'TRL', 'OP', 'IS', 'SR', 'RR', 'FLYOVER', 'STCR', 'ET', 'RN', 'RP', 'WKWY']

In [47]:
gdf.shape

(27891, 38)

In [48]:
gdf = gdf.loc[-gdf['ord_street_type'].isin(street_type_to_remove), :].copy()

In [49]:
gdf.shape

(26881, 38)

In [50]:
gdf['ord_street_type'].unique()

array(['AVE', 'ST', 'PL', 'LN', 'RD', 'WAY', 'PKWY', 'DR', 'CT', 'BLVD',
       'CIR', 'TER', 'BR', 'VIEW', 'LOOP', 'PZ', 'MALL'], dtype=object)

# SAVE AS A GDF - OVERWRITE

In [51]:
# file path
output_file_name = 'Street_Network_Database_Seattle_working.gpkg'

In [52]:
ofpn = os.path.join(file_path, output_file_name)

In [53]:
gdf.to_file(filename = ofpn, driver = 'GPKG', index = False)